# Use Bootstrap Aggregating
* Train each classifier on a subset the WSIs.Not true bagging because no replacement. Can't be true, because there isn't a fixed dateset.
* Try training each classifier on half, quarter of the dataset.

In [1]:
import os
import sys
import glob
import random
import pickle
import numpy as np
from PIL import Image
import time
import copy
from io import BytesIO
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data
import torchvision.models as models
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

SEED = 101
np.random.seed(SEED)
from torchvision import datasets, models, transforms

%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Add the src directory for functions
src_dir = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'src')
print(src_dir)
sys.path.append(src_dir)

# import my functions:
from WSI_pytorch_utils import*

fast_ai_dir = '/media/rene/Data/fastai/'
sys.path.append(fast_ai_dir)

# Set it to use GPU0
torch.cuda.set_device(0)
print(torch.cuda.is_available())
print(torch.cuda.current_device())

/media/rene/Data/camelyon/src
True
0


In [2]:
data_loc = '/media/rene/Data/CAMELYON16/TrainingData'

ttv_split = np.load('/media/rene/Data/camelyon/other/ttv_split.p')
normal_valid = ttv_split['normal_vaild_idx']
tumor_valid = ttv_split['tumor_vaild_idx']

normal_train = list(range(1, 161))
normal_train = [num for num in normal_train if num not in normal_valid]
tumor_train = list(range(1, 111))
tumor_train = [num for num in tumor_train if num not in tumor_valid]

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

dataset_sizes = {}
dataset_sizes['train'] = 10000
dataset_sizes['valid'] = 1000

batch_size = 64
epochs = 10
save_path = '/media/rene/Data/camelyon_out/inline_batch/resnet50_5_'

## Train the classifiers

In [3]:
for i in range(5):
    normal_train_subset = random.sample(normal_train, int(len(normal_train)/2))

    train_dataset = WSIDataset(data_loc, normal_train_subset, tumor_train, batch_size, length=dataset_sizes['train'], transforms=data_transforms['train'])
    valid_dataset = WSIDataset(data_loc, normal_valid, tumor_valid, batch_size, length=dataset_sizes['valid'], transforms=data_transforms['train'])

    dataloaders ={}
    dataloaders['train'] = DataLoader(train_dataset,  batch_size=1, num_workers=4, shuffle=False)
    dataloaders['valid'] = DataLoader(train_dataset, batch_size=1, num_workers=4, shuffle=False)


    model_ft = models.resnet50(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)
    model_ft = model_ft.cuda()

    criterion = nn.CrossEntropyLoss()

    # all parameters are being optimized
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=4, gamma=0.1)

    model_ft = train_model(model_ft, dataloaders, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=epochs)
    
    torch.save(model_ft.state_dict(), save_path+i)

RuntimeError: cuda runtime error (46) : all CUDA-capable devices are busy or unavailable at /opt/conda/conda-bld/pytorch_1518244421288/work/torch/lib/THC/generic/THCStorage.cu:58

In [ ]:
### CHECK SOME RANDOM SAMPLES FROM BATCH GEN
